In [34]:
!pip install opencv-python
!pip install matplotlib
!pip install IPython
!pip install opencv-contrib-python --user
!pip install tabulate

In [35]:
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import Image
from tabulate import tabulate

## Prepare image

In [3]:
# Load image from folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
plt.figure(figsize=(16,8))


<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

## Local Feature Extraction

In [5]:
## SIFT
def draw_keypoints_sift(keypoints,image,image_gray):
    final_image=cv2.drawKeypoints(image_gray,keypoints,image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(final_image)
    
def sift_local_feature_extraction(img):
    sift = cv2.SIFT_create() 
    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # find keypoints, descriptor
    keypoints, descriptors = sift.detectAndCompute(image_gray,None) 
    #draw_keypoints_sift(keypoints,img,image_gray)
    dic = dict()
    dic['kp'] = keypoints
    dic['des'] = descriptors
    return dic
#templates = load_images_from_folder('./templates/uno')
#images = load_images_from_folder('./images/uno')
#plt.imshow(cv2.cvtColor(templates[0], cv2.COLOR_BGR2RGB))
#template_kp_des = sift_local_feature_extraction(templates[0])
#image_kp_des = sift_local_feature_extraction(images[0])
#template_kp_des

## Image Matching

In [39]:
# FLANN(KD_TREE)
def flann_image_matching(img1,img2,kp1,kp2,des1,des2,FLANN_INDEX_KDTREE=0,distance_ratio = 0.7,MIN_MATCH_COUNT=50):
    return_dic = dict()
    return_dic['FLANN_INDEX_KDTREE'] = FLANN_INDEX_KDTREE
    return_dic['distance_ratio'] = distance_ratio
    return_dic['MIN_MATCH_COUNT'] = MIN_MATCH_COUNT
    
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    
    c_img2 = img2.copy()
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    matches = flann.knnMatch(des1,des2,k=2) # trả về 2 features gần nhất từ tập des2
    matchesMask = [[0,0] for i in range(len(matches))]
    
    good = []
    for m,n in matches:
        if m.distance < distance_ratio*n.distance: 
            # chỉ giữ lại những ghép cặp ổn định (m.distance: khoảng cách gần nhất, n.distance: khoảng cách gần thứ 2)
            good.append(m)
    return_dic['good'] = len(good)
    ## khoanh vùng đối tượng tương ứng được tìm thấy
    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        
        # tìm phép biến đổi đồng dạng giữa 2 tập điểm được so khớp
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0) 
    
        matchesMask = mask.ravel().tolist()
        h,w,d = img1.shape
    
        # xác định vị trí tương ứng trên ảnh đích của 4 góc trên ảnh nguồn
    
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2) # 4 điểm khoanh vùng trên ảnh nguồn
        dst = cv2.perspectiveTransform(pts,M) # thực hiện biến đổi để tìm ra vị trí tương ứng trong ảnh được so khớp
    
        # hiển thị kết quả tương ứng
        #print( "Matches are found - {}".format(len(good)) )
        c_img2 = cv2.polylines(c_img2,[np.int32(dst)],True,255,2, cv2.LINE_AA)
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
    ## hiển thị kết quả
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    img12 = cv2.drawMatches(img1,kp1,c_img2,kp2,good,None,**draw_params)
    
    return return_dic
    #plt.figure(figsize=(8,8))
    #plt.imshow(cv2.cvtColor(img12, cv2.COLOR_BGR2RGB))
    #plt.show()
    
    #plt.figure(figsize=(8,8))
    #plt.subplot(121)
    #plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    #plt.subplot(122)
    #plt.imshow(cv2.cvtColor(c_img2, cv2.COLOR_BGR2RGB))
    #plt.show ()

#flann_image_matching(templates[0],images[0],template_kp_des['kp'],image_kp_des['kp'],template_kp_des['des'],image_kp_des['des'])

## Object Detection 

In [50]:
# run with: toy, thaprua, uno
templates = load_images_from_folder('./templates/uno')
images = load_images_from_folder('./images/uno')

# Implementation
def object_detection(templates,images):
    col_names = ["Template","FLANN_INDEX_TREE", "Distance_Ratio", "GOOD"]
    for image in images:
        i = 1
        for template in templates:
            lst = []
            template_name = "template " + str(i)
            template_kp_des = sift_local_feature_extraction(template)
            image_kp_des = sift_local_feature_extraction(image)
            FLANN_INDEX_KDTREE = 0
            step_FLANN_INDEX_KDTREE = 1
            for n in range(6):
                distance_ratio = 0.1
                step_distance_ratio = 0.1
                while distance_ratio < 1:
                    dic = flann_image_matching(template,image,template_kp_des['kp'],image_kp_des['kp'],
                                 template_kp_des['des'],image_kp_des['des'],n,distance_ratio,30)
                    lst.append([template_name,dic['FLANN_INDEX_KDTREE'],dic['distance_ratio'],dic['good']])
                    distance_ratio += step_distance_ratio
                
            content = tabulate(lst, headers=col_names)
            print(content)
            print('\n')
            i+=1
            
            

object_detection(templates,images)        

Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 2/30
Not enough matches are found - 5/30
Not enough matches are found - 16/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 2/30
Not enough matches are found - 7/30
Not enough matches are found - 16/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 2/30
Not enough matches are found - 6/30
Not enough matches are found - 16/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 2/30
Not enough matches are found - 5/30
Not enough matches are found - 16/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are found - 0/30
Not enough matches are f